In [77]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

from scipy import optimize
from sklearn.metrics import mean_squared_error
from functools import partial
from tqdm import tqdm

# Surplus Yield Models

## Population Growth

### Reproductive Stability
Let's start with what feels like a pretty minor assumption - **Reproductive Stability**. If the fecund population (population that can produce offspring) is given by $P_F$ and the offspring in a year by $P_O$ then this assumption can be stated as:

$$P_O = \gamma P_F$$

Which is to say the offspring are a constant multiplier of the fecund population, each and every year.

### Demographic Stability

Now in order for us to model population growth generally we'd need to also model the transformation of offspring into fecund population. However in the case of what we'll call **Demographic Stability** we don't have to worry about this. What is this demographic stability? It's the idea that the proportion of the population that is fecund always remains the same.

$$P_F = \alpha P$$

Where $P$ is the overall population size.

What's nice about this assumption is that we can directly model population growth with:

$$P_{t+1} = P_{t} + \gamma \alpha P_{t}$$

No need to keep the two populations (fecund and immature) separate. 

### Biomass Stability

Another more practical assumption is that:

$$P=\beta B$$

Or that the population and biomass are related by a constant $\beta$. This is usefull because often times fisheries don't report the number of fish caught but the weight of those fish. Imagine trying to count individual shrimp or herring and you can get a sense of why this might be. 

With all of these assumptions in place we get:

$$ B_{t+1} = B_{t} + \gamma \alpha \beta B_{t} $$

Or just

$$ B_{t+1} = B_{t} + r B_{t} $$

In [78]:
B = [np.ones(3) * 0.1]
R = np.array([0.2, 0.3, 0.4])
rows = []
for t in range(1, 20 + 1):
    B.append(
        B[-1] + R * B[-1]
    )
    for r, b in zip(R, B[-1]):
        rows.append({
            'biomass': b,
            'time': t,
            'growth_ratio': r 
        })
px.line(
    pd.DataFrame(rows), x='time', y='biomass', color='growth_ratio'
)

In [79]:
fig = px.line(
    pd.DataFrame(rows), x='time', y='biomass', color='growth_ratio',
    color_discrete_map={
                   0.2: "rgba(1,1,1,0.33)",
                   0.3: "rgba(1,1,1,0.5)",
                   0.4: "rgba(1,1,1,1)"
               }
)

fig.update_traces(marker=dict(color='black'))
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
with open('SurplusModels/exponential_growth.png', 'wb') as fh:
    fh.write(fig.to_image('png'))
fig

## Population Control

Alright the population is clearly growing exponentially - however we know populations eventually reach their limits. Specifically there's a notion of the **Carrying Capacity** of an ecosystem which is how much of a specific species an ecosystem can actually support. Then the idea is that as a population reaches its carrying capacity it becomes harder and harder for the population to actually grow. What's driving this slow down? In an unfished system the answer is natural mortality which, given our demographic stability assumption must occur across the board. So how do we want to model this? 

### Linear Competition Pressure

One way to think about this is apply some kind of pressure to our $rB$ term from our population growth so that when the population is first growing $r$ is unlimited, but as the population grows $r$ effectively shrinks until it eventually becomes $0$ at $B_{\infty}$ (the carrying capacity). There are obviously loads of ways this pressure could manifest, however we can make the simplest assumption - **Linearity**. This would be acheived by multiplying $r$ by the following term:

$$ 1 - B/B_{\infty}$$

to get:

$$ B_{t+1} = B_{t} + r(1-B_t/B_{\infty}) B_{t} $$

In [74]:
B = [np.ones(3)*0.1]
B_max = 5
rows = []
for t in range(1, 20 + 1):
    nB = (B[-1] + R * (1 - B[-1]/B_max) * B[-1])
    nB[nB > B_max] = B_max
    B.append(nB)
    for r, b in zip(R, B[-1]):
        rows.append({
            'biomass': b,
            'time': t,
            'growth_ratio': r 
        })
px.line(
    pd.DataFrame(rows), x='time', y='biomass', color='growth_ratio'
)

In [76]:
fig = px.line(
    pd.DataFrame(rows), x='time', y='biomass', color='growth_ratio',
    color_discrete_map={
                   0.2: "rgba(1,1,1,0.33)",
                   0.3: "rgba(1,1,1,0.5)",
                   0.4: "rgba(1,1,1,1)"
               }
)

fig.update_traces(marker=dict(color='black'))
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
with open('SurplusModels/capped_growth.png', 'wb') as fh:
    fh.write(fig.to_image('png'))
fig

## Adding the Humans

Alright now that we've removed the population explosion it's time to add our catch $C$ in. Put most simply we just have:


$$ B_{t+1} = B_{t} + r(1-B_t/B_{\infty}) B_{t} - C_{t}$$

Let's play around with this catch rate for a bit. 

First suppose that we decide to catch nothing $C_{t}=0$.

In [80]:
B = 0.1
B_max = 5
r = 0.5
rows = []
for t in range(1, 20 + 1):
    B = min(B + r * (1 - B / B_max) * B, B_max)
    rows.append({
        'biomass': B,
        'time': t
    })
px.line(
    pd.DataFrame(rows), x='time', y='biomass'
)

Alright so after around 18 timesteps we get to $B_{\infty}$. So now let's suppose we're at carrying capacity and we start making a constant catch. What does this look like?

In [81]:
a = np.array([1, 2, 3])
a[a > 2] = 2

In [82]:
B = [np.ones(3)*B_max]
C = [0.1, 0.6, 0.75]
rows = [
    {
        'biomass': B_max,
        'time': 0,
        'catch': c
    } for c in C
]
for t in range(1, 20 + 1):
    nB = (B[-1] + r * (1 - B[-1]/B_max) * B[-1] - C)
    nB[nB > B_max] = B_max
    nB[nB < 0] = 0
    B.append(nB)
    for b, c in zip(B[-1], C):
        rows.append({
            'biomass': b,
            'time': t,
            'catch': c
        })
px.line(
    pd.DataFrame(rows), x='time', y='biomass', color='catch'
)

In [83]:
fig = px.line(
    pd.DataFrame(rows), x='time', y='biomass', color='catch',
    color_discrete_map={
                   0.1: "rgba(1,1,1,0.33)",
                   0.6: "rgba(1,1,1,0.5)",
                   0.75: "rgba(1,1,1,1)"
               }
)

fig.update_traces(marker=dict(color='black'))
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
with open('SurplusModels/adding_the_humans.png', 'wb') as fh:
    fh.write(fig.to_image('png'))
fig

This is very interesting! At a very low catch rate $C=0.1$ we see the population barely moves from its $B_{\infty}$. As we raise the catch to $0.6$ we see the population initially dives but then stabilizes at $\approx 3$. Then as we cross over to $C=0.75$ the population just ends up plumetting. 

To see what's driving this let's look at the growth of the population as a function of biomass $B$.

In [70]:
B = np.arange(0, B_max, 0.01)
G = r * (1 - B / B_max) * B
px.line(
    pd.DataFrame(np.array([B, G]).T, columns=['biomass', 'growth']), x='biomass', y='growth'
)

In [71]:
fig = px.line(
    pd.DataFrame(np.array([B, G]).T, columns=['biomass', 'growth']), x='biomass', y='growth'
)
fig.update_traces(line_color='black', marker=dict(color='black'))
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
with open('SurplusModels/surplus_yield.png', 'wb') as fh:
    fh.write(fig.to_image('png'))
fig

From this we can see that the growth is maximal not when the population is halfway to $B_{\infty}$. This however makes sense as we have the two competing (and linear) forces of population growth and carrying capacity pressure. 

So how does this relate to our curves of population change under varying catch rates above? Well in each case because $C > 0$ the population took a hit because the growth at $B=B_{\infty}$ is $0$. However as the population shrunk, this relieves the carrying capacity pressure thereby allow the growth rate to increase. For the lower catch rates the growth rate eventually exceeded the catch rate bringing us to a point of stability. But for $C=0.75$ the growth rate could never exceed the catch rate and so eventually we drove the population past its midpoint and then the growth rate began to plumet as well sending the species straight for extinction. 

This, by the way is where the **Surplus Yield** model name comes from - the idea is to model a fishery that only takes "new" growth instead of exceeding the "surplus production" of the fishery and therefore driving it to extinction. 

The tricky bit is that if you actually "want" to drive the population back to its halfway point in order to take the maximum sustainable yield or MSY. Here's a full set of catch rate curves to show you exactly that. How we go from sustainable small yields to sustainable large yields to unsustainable yields.

In [8]:
C = np.arange(0, B_max, 0.1)
B = [np.ones(C.shape[0])*B_max]
rows = [
    {
        'biomass': B_max,
        'time': 0,
        'catch': c
    } for c in C
]
for t in range(1, 20 + 1):
    nB = (B[-1] + r * (1 - B[-1]/B_max) * B[-1] - C)
    nB[nB > B_max] = B_max
    nB[nB < 0] = 0
    B.append(nB)
    for b, c in zip(B[-1], C):
        rows.append({
            'biomass': b,
            'time': t,
            'catch': c
        })
px.line(
    pd.DataFrame(rows), x='time', y='biomass', color='catch'
)

So with all this in mind what's the MSY?

$$ rB_{\infty}/4$$

## The Problem of Fitting

Alright so we've got this lovely formula for the MSY.

$$C_{MSY} = rB_{\infty}/4$$

However there's one small problem. How on earth do we figure out what $r$ and $B_{\infty}$ actually are? This brings us to the problem of fitting our data.

Fitting our data means finding a series of things we can measure and then pitting them against the things we can't measure but hope to "fit" to the data. Our unmeasurables are clearly $r$ and $B$ but what are our measureables? Catch. However here is where we immediately run into a problem. Let's look at our model thus far:

$$ B_{t+1} = B_{t} + r(1-B_t/B_{\infty}) B_{t} - C_{t}$$


 As we've stated things thus far $C$ is a free variable - while how much we catch certainly has to obey the constraint of $C_{t} < B_{t+1}$ beyond that we have no relationship between catch and the biomass. Put another way, as we've stated the problem at present biomass and our growth rate $r$ don't predict the catch $C$ at all... so there's no measureable to relate to an unmeasureable... 

Time it seems, for another assumption, one to connect catch to biomass so we can tie these things together and start predicting something. 

### Linear Relationship Between Biomass and Catch Effort

To bring in this new assumption we have to introduce a new concept - catch per unit effort (CPUE). This idea is pretty simple. It's how much catch (usually in biomass) people end up with a certain amount of effort. What "effort" is is kind of irrelevant so long as it is consistent. So it could be miles of net, number of pots, number of hooks, hours at sea, etc. 

Okay so our assumption is this:

$$CPUE = qB$$

which if the effort input is $f$ we have:

$$C = f\bullet CPUE = fqB$$

Alright so how does this help us? We've just connected catch to biomass! Given a predicted biomass, q, and effort (which is also typically measured) we can predict what the catch would've been. As such our model now becomes:

$$ B_{t+1} = B_{t} + r(1-B_t/B_{\infty}) B_{t} - qfB_{t}$$

Note what's important is that catch $C$ is not a variable in this, but is now something that can be computed one this equation is run. In other words we have a value we can predict without explicitly putting in the model - and this will allow us to fit our parameters.

Now specifically we have $B_{i}$ which would be the biomass measured at the beginning of our measurements on catch and then $r$, $B_{\infty}$, and $q$ as free variables. So we have four parameters to tune and therefore need at least four years to fit the data to. 

Specifically for some number of measurements we can represent this as a function that takes four parameters and produces an equal number of catch predictions. Then we can just use normal ML procedures for fitting to the data. 

In [9]:
def predict_biomasses(B_i, B_max, r, q, F):
    biomasses = [B_i]
    for f in F[:-1]: # we need only predict F.shape[0] - 1 biomasses
        B = biomasses[-1]
        biomasses.append(
            min(B + r * (1 - B / B_max) * B - q * f * B, B_max)
        )
    return np.array(biomasses)

def predict_catches(B, q, F):
    return q * F * B


So the from "Fisheries Biology, Assessment and Management" we get the following example data:

In [10]:
C = np.array([432, 382, 431, 656, 594, 508, 505, 534, 593, 764, 532, 632, 1054, 607, 856, 820, 745])
F = np.array([15700, 17300, 21000, 22800, 31300, 34900, 41400, 45000, 66300, 66900, 71400, 71400, 72000, 85500, 95400, 95900, 100700])
fisheries_data = pd.DataFrame(np.array([C, F]).T, columns=['catch', 'effort'])
fisheries_data['case'] = 'actuals'
px.scatter(fisheries_data, x='effort', y='catch')

In [11]:
fig = px.scatter(fisheries_data, x='effort', y='catch')
fig.update_traces(line_color='black', marker=dict(color='black'))
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
with open('SurplusModels/measurements.png', 'wb') as fh:
    fh.write(fig.to_image('png'))
fig

We can start by using the values they arrived at:

In [12]:
B_i, B_max = 4126, 4126
r, q = 0.50318, 5.7 * 10 ** -6

book_pred_C = predict_catches(
    predict_biomasses(B_i, B_max, r, q, F), q, F
)
book_predicted = pd.DataFrame(np.array([book_pred_C, F]).T, columns=['catch', 'effort'])
book_predicted['case'] = 'book'
fisheries_data = pd.concat([book_predicted, fisheries_data])
px.scatter(fisheries_data, x='effort', y='catch', color='case')

Next we'll go ahead and try to fit the data starting from the same starting point the book used.

In [47]:
def evaluate_model(C, F, x, *args):
    s, B_max, r, q = x
    B_i = s * B_max
    B = predict_biomasses(B_i, B_max, r, q, F)
    pred_C = predict_catches(B, q, F)
    return mean_squared_error(
        C, pred_C
    )

s, B_max, r, q = optimize.minimize(
    partial(evaluate_model, C, F), np.array([1, 4000, 0.5, 0.000005]),  method='Nelder-Mead', bounds=[(0.1, 1), (0, 10000), (10 ** -5, 0.9), (10 ** -10, 0.9)]
).x
B_i = s * B_max

nb_pred_C = predict_catches(
    predict_biomasses(B_i, B_max, r, q, F), q, F
)
print('B_i =', B_i, 'B_max =', B_max, 'r =', r, 'q =', q)
print('Book Mean Squared Error:', mean_squared_error(C, book_pred_C))
print('Notebook Mean Squared Error:', mean_squared_error(C, nb_pred_C))
nb_predicted = pd.DataFrame(np.array([nb_pred_C, F]).T, columns=['catch', 'effort'])
nb_predicted['case'] = 'predicted'
overall_data = pd.concat([book_predicted, fisheries_data, nb_predicted])
px.scatter(overall_data, x='effort', y='catch', color='case')

B_i = 7457.609247093652 B_max = 4635.1704858991 r = 0.5268085989596984 q = 3.673951952093616e-06
Book Mean Squared Error: 42615.68736402773
Notebook Mean Squared Error: 24634.030178519486


/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.



In [53]:
fig = px.scatter(
    overall_data[overall_data['case'].isin(['actuals', 'predicted'])], x='effort', y='catch', symbol='case',
    symbol_map={'predicted': 'cross', 'actuals': 'circle'}
)
fig.update_traces(line_color='black', marker=dict(color='black'))
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
with open('SurplusModels/model_scatter.png', 'wb') as fh:
    fh.write(fig.to_image('png'))
fig

In [61]:
df = nb_predicted.merge(
    fisheries_data[fisheries_data['case'] == 'actuals'], 
    on='effort', suffixes=('_predicted', '_actuals')
)
df['abs_error'] = abs(df['catch_predicted'] - df['catch_actuals'])
fig = ff.create_distplot([df['abs_error']], ['Absolute Error'], show_hist=False)
fig.update_traces(line_color='black', marker=dict(color='black'))
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
with open('SurplusModels/model_dist.png', 'wb') as fh:
    fh.write(fig.to_image('png'))
fig


Okay so we have better error! 

Remembering that:

$$C_{MSY} = rB_{\infty}/4$$

Let's see what the difference in $C_{MSY}$'s are between the two models.

In [14]:
book_msy = round(0.50318 * 4126 / 4)
notebook_msy = round(r * B_max / 4)
print("Book's MSY:", book_msy)
print("Notebook's MSY:", notebook_msy)
print("Notebook/Book", round(notebook_msy / book_msy, 2))

Book's MSY: 519
Notebook's MSY: 610
Notebook/Book 1.18


So by finding a better fit we just increased the sustainable catch by nearly 30%! How on earth is this possible?

## Sensitivity Analysis

One question that should immediately arise is how accurate are all of these parameters and more importantly how accurate do we actually expect this $C_{MSY}$ to be? To answer this we're going to have to see how wobbly our parameters can get and we'll do so with two different methods.

1. Noise in the Data: put noise into our actuals and see how wobbly our $C_{MSY}$ becomes. 
2. Variation in the Parameters: we'll introduce small variations to parameters, retune and see how much the $C_{MSY}$ changes

### Noise in the Data

In [25]:
starting_guess = np.array([1, 4000, 0.5, 0.000005])
trials = 100
max_error_C = 0.1
max_error_F = 0.1
rows = []
for i in tqdm(range(trials)):
    noisy_C = C + C * ((np.random.random(C.shape[0]) - 0.5) * 2 * max_error_C)
    noisy_F = F + F * ((np.random.random(F.shape[0]) - 0.5) * 2 * max_error_F)
    s, B_max, r, q = optimize.minimize(
        partial(evaluate_model, noisy_C, noisy_F), starting_guess,  method='Nelder-Mead', bounds=[(0.1, 1), (0, 10000), (10 ** -5, 0.9), (10 ** -10, 0.9)]
    ).x
    B_i = s * B_max

    pred_noisy_C = predict_catches(
        predict_biomasses(B_i, B_max, r, q, noisy_F), q, noisy_F
    )
    rows.append({
        'trial': i,
        'B_i': B_i,
        'B_max': B_max,
        'r': r,
        'q': q,
        'B_msy': r * B_max / 4,
        'MSE': mean_squared_error(noisy_C, pred_noisy_C)
    })
noisy_df = pd.DataFrame(rows)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  2%|▏         | 2/100 [00:00<00:05, 16.99it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  4%|▍         | 4/100 [00:00<00:06, 13.76it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraint

In [26]:
print(np.percentile(noisy_df['B_msy'], [5, 25, 50, 75, 95]))
px.histogram(noisy_df, x='B_msy', nbins=int(trials/5))

[513.99029482 568.16498225 591.95965303 611.8374729  640.87466407]


In [35]:
fig = ff.create_distplot([noisy_df['B_msy']], ['MSY'], show_hist=False)
fig.update_traces(line_color='black', marker=dict(color='black'))
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
with open('SurplusModels/instrumentation.png', 'wb') as fh:
    fh.write(fig.to_image('png'))
fig

So in this case where we've assumed up to a 10% error in both the catch and effort we have a 5th percentile $C_{MSY}$ of 531, a median of 637, and a 95th percentile of 683. So up to 15% below the median or up to 7% above (this distribution is quite skewed). $\approx 11$ percent error overall. 

In [17]:
starting_guess = np.array([1, 4000, 0.5, 0.000005])
trials = 100
max_error_C = 0.2
max_error_F = 0.2
rows = []
for i in tqdm(range(trials)):
    noisy_C = C + C * ((np.random.random(C.shape[0]) - 0.5) * 2 * max_error_C)
    noisy_F = F + F * ((np.random.random(F.shape[0]) - 0.5) * 2 * max_error_F)
    s, B_max, r, q = optimize.minimize(
        partial(evaluate_model, noisy_C, noisy_F), starting_guess,  method='Nelder-Mead', bounds=[(0.1, 1), (0, 10000), (10 ** -5, 0.9), (10 ** -10, 0.9)]
    ).x
    B_i = s * B_max

    pred_noisy_C = predict_catches(
        predict_biomasses(B_i, B_max, r, q, noisy_F), q, noisy_F
    )
    rows.append({
        'trial': i,
        'B_i': B_i,
        'B_max': B_max,
        'r': r,
        'q': q,
        'B_msy': r * B_max / 4,
        'MSE': mean_squared_error(noisy_C, pred_noisy_C)
    })
noisy_df = pd.DataFrame(rows)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  2%|▏         | 2/100 [00:00<00:08, 11.17it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  4%|▍         | 4/100 [00:00<00:06, 14.70it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraint

In [18]:
print(np.percentile(noisy_df['B_msy'], [5, 25, 50, 75, 95]))
px.histogram(noisy_df, x='B_msy', nbins=int(trials/5))

[379.40941705 478.60898955 539.5441321  609.33907978 719.21042238]


With up to 20% error we get 5th percentile as 395, median as 590, and 95th as 740 or up to 33% below and 25% above which is more like a 30% error overall.

### Noise in the Parameters - $r$

In [21]:
trials = 100
max_error = 0.2
base_error = 26492.019262714537
base_r = 0.8514128788449058
R = (((np.random.random(trials) - 0.5) * 2 * max_error) + 1) * base_r
rows = []
for r in tqdm(R):
    s, B_max, r, q = optimize.minimize(
        partial(evaluate_model, C, F), np.array([1, 4000, r, 0.000005]),  method='Nelder-Mead', bounds=[(0.1, 1), (0, 10000), (r, r), (10 ** -10, 0.9)]
    ).x
    B_i = s * B_max

    pred_noisy_C = predict_catches(
        predict_biomasses(B_i, B_max, r, q, noisy_F), q, noisy_F
    )
    mse = mean_squared_error(noisy_C, pred_noisy_C)
    if abs(base_error - mse) / base_error <= 0.1:
        rows.append({
            'trial': i,
            'B_i': B_i,
            'B_max': B_max,
            'r': r,
            'q': q,
            'B_msy': r * B_max / 4,
            'MSE': mse
        })
noisy_params_df = pd.DataFrame(rows)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  2%|▏         | 2/100 [00:00<00:08, 11.88it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  4%|▍         | 4/100 [00:00<00:07, 13.31it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraint

In [23]:
noisy_params_df

""


In [22]:
px.histogram(noisy_params_df, x='MSE')

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of [] but received: MSE

In [ ]:
print(np.percentile(noisy_params_df['B_msy'], [5, 25, 50, 75, 95]))
px.histogram(noisy_params_df, x='B_msy')

[610.45591174 610.46122253 610.46126084 610.46198661 610.50593899]


In this case we got our 5th percentile as 640, median as 662, and 95th as 676 or up to 3% below and 2% above which is more like a 3% error overall. Much more stable for $r$.

### Noise in the Parameters - $B_{\infty}$

In [ ]:
trials = 100
max_error = 0.2
base_error = 26492.019262714537
base_B_max = 3102.4030498576967
B_MAX = (((np.random.random(trials) - 0.5) * 2 * max_error) + 1) * base_B_max
rows = []
for B_max in tqdm(B_MAX):
    s, B_max, r, q = optimize.minimize(
        partial(evaluate_model, C, F), np.array([1, B_max, 0.5, 0.000005]),  method='Nelder-Mead', bounds=[(0.1, 1), (B_max, B_max), (10 ** -5, 0.9), (10 ** -10, 0.9)]
    ).x
    B_i = s * B_max

    pred_noisy_C = predict_catches(
        predict_biomasses(B_i, B_max, r, q, noisy_F), q, noisy_F
    )
    mse = mean_squared_error(noisy_C, pred_noisy_C)
    if abs(base_error - mse) / base_error <= 0.1:
        rows.append({
            'trial': i,
            'B_i': B_i,
            'B_max': B_max,
            'r': r,
            'q': q,
            'B_msy': r * B_max / 4,
            'MSE': mse
        })
noisy_params_df = pd.DataFrame(rows)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  1%|          | 1/100 [00:00<00:09,  9.96it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  2%|▏         | 2/100 [00:00<00:11,  8.72it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

  4%|▍         | 4/100 [00:00<00:08, 11.58it/s]/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarning:

Method Nelder-Mead cannot handle constraints nor bounds.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_minimize.py:534: RuntimeWarnin

In [ ]:
px.histogram(noisy_params_df, x='MSE')

In [ ]:
print(np.percentile(noisy_params_df['B_msy'], [5, 25, 50, 75, 95]))
px.histogram(noisy_params_df, x='B_msy')

[610.44031055 610.46111258 610.46124503 610.46145794 610.48659628]


In this case we got our 5th percentile as 601, median as 648, and 95th as 663 or up to 7% below and 2% above which is more like a 5% error overall.

## What to Make of All of This

### Ensuring No Overestimate

With our better fit we got $C_{MSY}=660$ however as we've seen the error involved here can easily be 5% from parameter noise and (for 10% error to $C$ and $F$) around 11% for data inaccuracies. Obviously we should assume the worst (otherwise extinction is around the corner) which means we should probably reduce our estimate by 15-20% which brings us to $C_{MSY}=528$ as an estimate that includes issues in our modeling. In some sense this is our $p<0.05$ value for $C_{MSY}$ to ensure that it is unlikely that we are *overestimating* the actual $C_{MSY}$ value.

### Allowing Buffer

However there's another problem. What if the biomass is exactly midway to $B_{\infty}$ and the stock has a bad year? Well then the growth from the diminished stock won't be able to keep pace with $C_{MSY}$ and we'll then drive the stock to extinction. So we really need to provide some buffer. Looking at our growth rate vs biomass curve from above we can see that we could loose 20% of the $B_{MSY}$ and the growth rate would drop by just over 3%. So if we were to allot for that 3% we'd give the stock a chance to drop 20% and still recover which seems reasonable. 

This then brings us to a final answer of:

$$C_{MSY} = 512$$


## Conclusions

As we've seen surplus yield models are pretty easy to understand and fit. They also require pretty minimal data - something that's great if a fishery is new or one that hasn't been paid much attention to. However this ease of use comes at the cost of some pretty hefty assumptions. 

For one thing we've assumed this whole way through a lot of stability that in reality probably doesn't exist. Fishing gear is selective - it doesn't sample the population indiscriminately. Carrying capacity is not a hard minimum and populations often boom and bust instead of nicely scaling up to some largest size. Reproductive success depends on much more than just the overall size of the population and even natural mortality is not indiscriminate with regards to age. Furthermore selection pressure means that for species who do reproduce quickly and in large quantities they can actually adapt to the fishing practices out there.

And then there's the fact that our whole $CPUE=qB$ assumption more or less entails the idea that catch effort doesn't change with time. Yet the data we just fit to would've been taken over *17 years*! That's plenty of time for new fishing tactics or technology to come into the picture and make $q$ get higher and higher. 

Finally we've just seen how much variance exists just in the modeling process and the accuracy of the data collected itself. Given things like bycatch and the vaguery of something like "fishing effort" we could easily imagine scenarios where the error margins on these $C_{MSY}$ values get up to 30+ percent! That means 30 percent fewer fishers! 

Obviously then there are improvements that must be made to this model and new data that must be collected in order to deal with these issues. It's to those models we'll turn next.